In [2]:
import fxcmpy
TOKEN = "65aa373e2c6d310060a655ac665b46531ecd1adc"
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level="error")
con

In [445]:
import datetime as dt
from icecream import ic 


start = dt.datetime(2021,12,20)
end = dt.datetime(2021,12,31)
data = con.get_candles('EUR/USD', period='m1', number=250) #number=250 start=start, end=end
data

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
date,,,,,,,,,
2022-01-04 01:00:00,1.13051,1.13052,1.13056,1.13050,1.13063,1.13064,1.13068,1.13063,99
2022-01-04 01:01:00,1.13052,1.13062,1.13062,1.13052,1.13065,1.13073,1.13074,1.13064,70
2022-01-04 01:02:00,1.13061,1.13049,1.13071,1.13049,1.13073,1.13061,1.13083,1.13060,108
2022-01-04 01:03:00,1.13049,1.13051,1.13052,1.13049,1.13062,1.13063,1.13064,1.13062,16
2022-01-04 01:04:00,1.13051,1.13063,1.13063,1.13051,1.13064,1.13075,1.13076,1.13064,42
...,...,...,...,...,...,...,...,...,...
2022-01-04 05:05:00,1.13003,1.13005,1.13006,1.13002,1.13013,1.13017,1.13017,1.13013,20
2022-01-04 05:06:00,1.13004,1.13004,1.13004,1.13002,1.13017,1.13015,1.13017,1.13014,26
2022-01-04 05:07:00,1.13003,1.13009,1.13011,1.12999,1.13015,1.13022,1.13023,1.13012,75


In [201]:
import pandas as pd
def print_data(data, dataframe):
    print('%3d | %s | %s, %s, %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1], data['Rates'][2],
             data['Rates'][3]))

def arraySortedOrNot(arr):
 
    # Calculating length
    n = len(arr)
 
    # Array has one or no element or the
    # rest are already checked and approved.
    if n == 1 or n == 0:
        return True
 
    # Recursion applied till last element
    return arr[0] <= arr[1] and arraySortedOrNot(arr[1:])

In [171]:
# import ta
# from ta.trend import *
# from ta.momentum import *


# data["SMALL_SMA"]=sma_indicator(data.bidclose, window=7)
# data["LARGE_SMA"]=sma_indicator(data.bidclose, window=10)
# # Stochastic need to fix
# data["STOCH"] = stoch(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["STOCH_SIGNAL"] = stoch_signal(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["RSI"] = rsi(data.bidclose)
# data["SMA_SIGNAL"] = 

# len(data.index)

# data

In [456]:
import importlib
import strategy
importlib.reload(strategy)
from strategy import BaseStrategy
import pandas as pd
from ta.trend import *
from ta.momentum import *

class SMAStrategy(BaseStrategy):
    name = "smarsistochastic_strategy"
    description = """7,10 SMA, 7,3,3 Stochastic and RSI Strategy"""
    
    def signal_found(self, data:dict):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "SELL"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "BUY"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        
        df.loc[(df["sma_signal"] == "BUY")  & (df["stoch"] >= 60) & (df["stoch_signal"] >= 60), "stoch_signal_f"] = "CHUTIYA"
        df.loc[(df["sma_signal"] == "SELL")  & (df["stoch"] <= 40) & (df["stoch_signal"] <= 40), "stoch_signal_f"] = "CHUTIYA" 
        df["entry_signal"] = df.sma_signal
        df["entry_price"] = df.bidopen
        return self.signal_found(df.loc[(df["stoch_signal_f"]== "CHUTIYA")].tail(1))
        
    
    def apply_indicator(self):
        super(SMAStrategy, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["stoch"] = round(stoch(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["stoch_signal"] = round(stoch_signal(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 7)


    
# strategy = SMAStrategy()
# # for i in range(51,100):
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# df = strategy.get_signal()
# df = df.loc[(df["SMA_SIGNAL"]=="BUY") | (df["SMA_SIGNAL"]=="SELL")]


# df = df.loc[(df["STOCH_SIGNAL_F"] == "CHUTIYA" )]
# df = df.loc[((df["SMA_SIGNAL"] == "BUY") & (df["SMALL_SMA"] <= df.bidopen) & (df["SMALL_SMA"] <= df.bidclose) &
#            (df["LARGE_SMA"] <= df.bidopen) & (df["LARGE_SMA"] <= df.bidclose)) | 
#            ((df["SMA_SIGNAL"] == "SELL") & (df["SMALL_SMA"] >= df.bidopen) & (df["SMALL_SMA"] >= df.bidclose) &
#            (df["LARGE_SMA"] >= df.bidopen) & (df["LARGE_SMA"] >= df.bidclose))
#            ]
# df = df.loc[((df["SMA_SIGNAL"] == "BUY") & (df["RSI"] >= 70)) | ((df["SMA_SIGNAL"] == "SELL") & (df["RSI"] <= 30))]
# df = df.loc[:, ["bidopen","bidclose","SMA_SIGNAL", "SMALL_SMA", "LARGE_SMA", "RSI", "STOCH","STOCH_SIGNAL"]]
# df.drop(["askopen","askclose", "askhigh", "tickqty"], axis=1)
# df
# df.insert(loc = 0,column = 'Strategy Valid?',value = '<input type="checkbox" />')
# df.tail(50)
# df.style
# df

In [449]:
class SMAStrategy1(BaseStrategy):
    name = "smarsistochastic_strategy"
    description = """7,10 SMA, 7,3,3 Stochastic and RSI Strategy"""
    
    def signal_found(self, data:dict):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "UP_CROSSOVER"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "DOWN_CROSSOVER"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        df["entry_signal"] = df.sma_signal
        df["entry_price"] = df.bidopen
        return df
        
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        # self.dataframe["stoch"] = round(stoch(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        # self.dataframe["stoch_signal"] = round(stoch_signal(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        # self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 7)

strategy = SMAStrategy1()
strategy.dataframe = data
strategy.is_valid_dataframe()
dd = strategy.get_signal()
dd

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty,small_sma,large_sma,sma_signal,entry_signal,entry_price
date,,,,,,,,,,,,,,
2022-01-04 01:09:00,1.13051,1.13060,1.13061,1.13051,1.13064,1.13072,1.13073,1.13063,49,1.130584,1.130572,DOWN_CROSSOVER,DOWN_CROSSOVER,1.13051
2022-01-04 01:10:00,1.13060,1.13067,1.13074,1.13060,1.13073,1.13080,1.13087,1.13072,58,1.130607,1.130587,,,1.13060
2022-01-04 01:11:00,1.13067,1.13064,1.13076,1.13064,1.13079,1.13077,1.13089,1.13075,86,1.130609,1.130589,,,1.13067
2022-01-04 01:12:00,1.13065,1.13072,1.13076,1.13064,1.13077,1.13085,1.13088,1.13077,61,1.130609,1.130612,UP_CROSSOVER,UP_CROSSOVER,1.13065
2022-01-04 01:13:00,1.13072,1.13079,1.13081,1.13072,1.13084,1.13091,1.13093,1.13084,77,1.130634,1.130640,,,1.13072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-04 05:05:00,1.13003,1.13005,1.13006,1.13002,1.13013,1.13017,1.13017,1.13013,20,1.130039,1.130034,,,1.13003
2022-01-04 05:06:00,1.13004,1.13004,1.13004,1.13002,1.13017,1.13015,1.13017,1.13014,26,1.130044,1.130033,,,1.13004
2022-01-04 05:07:00,1.13003,1.13009,1.13011,1.12999,1.13015,1.13022,1.13023,1.13012,75,1.130049,1.130041,,,1.13003


In [ ]:
test_data = data.copy()
import backtest
importlib.reload(strategy)
importlib.reload(backtest)
from backtest import BackTestStrategy
importlib.reload(strategy)
importlib.reload(backtest)

sma_backtest = BackTestStrategy(SMAStrategy, test_data)
sma_backtest.start_backtesting()

In [453]:
kd = sma_backtest.get_report()

# kd["max_bid"] = kd[["bidopen","bidclose","bidlow","bidhigh"]].max(axis=1)2022-01-03 10:53:00
kd.drop(["askopen","askclose", "askhigh", "tickqty", "stoch_signal", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
        "small_sma", "large_sma","stoch"], axis=1)

,bidopen,rsi,sma_signal,stoch_signal_f,entry_signal,entry_price,max_bid,min_bid,exit_price,exit_time,status
2022-01-04 01:14:00,1.13079,69.548473,BUY,CHUTIYA,BUY,1.13079,1.13082,1.13076,1.13039,2022-01-04 01:35:00,SL
2022-01-04 01:53:00,1.13062,71.087839,BUY,CHUTIYA,BUY,1.13062,1.13063,1.13055,1.12998,2022-01-04 02:56:00,SL
2022-01-04 02:00:00,1.13060,51.833332,SELL,CHUTIYA,SELL,1.13060,1.13066,1.13053,1.12998,2022-01-04 02:56:00,TG
2022-01-04 02:23:00,1.13057,84.480621,BUY,CHUTIYA,BUY,1.13057,1.13076,1.13056,1.12998,2022-01-04 02:56:00,SL
2022-01-04 02:33:00,1.13069,72.980081,BUY,CHUTIYA,BUY,1.13069,1.13079,1.13069,1.12998,2022-01-04 02:56:00,SL
2022-01-04 02:49:00,1.13073,37.277029,SELL,CHUTIYA,SELL,1.13073,1.13076,1.13066,1.12998,2022-01-04 02:56:00,TG
2022-01-04 03:08:00,1.13003,33.550974,SELL,CHUTIYA,SELL,1.13003,1.13009,1.13000,1.12960,2022-01-04 03:24:00,TG
2022-01-04 03:31:00,1.12996,63.788469,BUY,CHUTIYA,BUY,1.12996,1.13004,1.12996,1.13044,2022-01-04 04:37:00,TG
2022-01-04 03:47:00,1.13000,59.864648,BUY,CHUTIYA,BUY,1.13000,1.13009,1.12996,1.12958,2022-01-04 04:00:00,SL
2022-01-04 03:49:00,1.12996,27.482447,SELL,CHUTIYA,SELL,1.12996,1.12996,1.12984,1.13044,2022-01-04 04:37:00,SL


In [145]:
d = d1.head(1)

# dt = data.index
# for dl in d1.itertuples():
#     print(dl)
pd.DataFrame().emptytarget_hit_temp

True

In [313]:
d1 = data.iloc[1]
d2 = data.iloc[10]
d2.bidopen

1.13492

In [381]:
all([False, False])

False

In [455]:
"bidopen" in  data.columns

True

In [363]:
a

[1, 9, 2, 3]